In [ ]:
import os 
import pandas as pd
import glob
import shutil

# Set base path to your ERCOT folder
base_path = r"C:\Users\amina.talipova\Desktop\ercot"
output_folder = os.path.join(base_path, "datasets")

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Define folders and output filenames
folders = {
    "AS": "AS_combined.csv",
    "DAM": "DAM_combined.csv",
    "RTM": "RTM_combined.csv"
}

def combine_csvs(folder_path):
    """Combine all CSVs in a folder into one DataFrame."""
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    df_list = []
    for file in all_files:
        try:
            df = pd.read_csv(file)
            df["source_file"] = os.path.basename(file)  # optional: track source
            df_list.append(df)
        except Exception as e:
            print(f"Skipping {file} due to error: {e}")
    if df_list:
        return pd.concat(df_list, ignore_index=True)
    else:
        return pd.DataFrame()

# Run the merging for each folder
for folder_name, output_file in folders.items():
    folder_path = os.path.join(base_path, folder_name)
    combined_df = combine_csvs(folder_path)
    if not combined_df.empty:
        output_path = os.path.join(output_folder, output_file)
        combined_df.to_csv(output_path, index=False)
        print(f"✅ Saved: {output_path}")
    else:
        print(f"⚠️ No CSV files found in: {folder_path}")




✅ Saved: C:\Users\amina.talipova\Desktop\ercot\datasets\AS_combined.csv
✅ Saved: C:\Users\amina.talipova\Desktop\ercot\datasets\DAM_combined.csv
✅ Saved: C:\Users\amina.talipova\Desktop\ercot\datasets\RTM_combined.csv


In [1]:
import os
import glob
import pandas as pd
import shutil

# Automatically load all CSV files from ESR folder
input_folder = r"C:\Users\amina.talipova\Desktop\ercot\ESR"
input_files = glob.glob(os.path.join(input_folder, "*.csv"))

# Load and combine
df_list = [pd.read_csv(file) for file in input_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Optional: remove duplicates
combined_df.drop_duplicates(inplace=True)

# Save to new file (same as before)
output_file = r"C:\Users\amina.talipova\Desktop\ercot\datasets\BESS_combined.csv"
combined_df.to_csv(output_file, index=False)
print(f"✅ Combined BESS data saved to:\n{output_file}")

# Original file path (same as before)
source_file = r"C:\Users\amina.talipova\Desktop\ercot\AL\20140224-20250430 ERCOT Actual Load.csv"

# Destination path and filename (same as before)
destination_folder = r"C:\Users\amina.talipova\Desktop\ercot\datasets"
os.makedirs(destination_folder, exist_ok=True)
destination_file = os.path.join(destination_folder, "AL_combined.csv")

# Copy and rename
shutil.copyfile(source_file, destination_file)
print(f"✅ Load file copied to: {destination_file}")


✅ Combined BESS data saved to:
C:\Users\amina.talipova\Desktop\ercot\datasets\BESS_combined.csv
✅ Load file copied to: C:\Users\amina.talipova\Desktop\ercot\datasets\AL_combined.csv


In [1]:
import pandas as pd
import os

# Paths
base_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets"
input_file = os.path.join(base_path, "RTM_combined.csv")
output_file = os.path.join(base_path, "RTM_2024_only.csv")

# Process in chunks
chunksize = 500_000
filtered_chunks = []

for chunk in pd.read_csv(input_file, parse_dates=["Date"], chunksize=chunksize):
    chunk_2024 = chunk[chunk["Date"].dt.year == 2024]
    filtered_chunks.append(chunk_2024)

# Combine and export
rtm_2024 = pd.concat(filtered_chunks, ignore_index=True)
rtm_2024.to_csv(output_file, index=False)

print(f"✅ Saved 2024-only RTM data to: {output_file}")


✅ Saved 2024-only RTM data to: C:\Users\amina.talipova\Desktop\ercot\datasets\RTM_2024_only.csv


In [9]:
import pandas as pd
import os
from openpyxl import load_workbook

# === 1. File list (excluding 2015) ===
fuel_mix_files = [
    "IntGenByFuel2016.xlsx",
    "IntGenbyFuel2017.xlsx",
    "IntGenbyFuel2018.xlsx",
    "IntGenbyFuel2019.xlsx",
    "IntGenbyFuel2020.xlsx",
    "IntGenbyFuel2021.xlsx",
    "IntGenbyFuel2022.xlsx",
    "IntGenbyFuel2023.xlsx",
    "IntGenbyFuel2024.xlsx",
    "IntGenbyFuel2025.xlsx"
]

# === 2. Folder with files ===
base_path = r"C:\Users\amina.talipova\Desktop\ercot\FM"

# === 3. Collect and parse all sheets ===
all_rows = []

def get_engine(filename):
    return 'openpyxl' if filename.endswith('.xlsx') else 'xlrd'

for file in fuel_mix_files:
    file_path = os.path.join(base_path, file)
    try:
        wb = load_workbook(file_path, read_only=True)
        for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                      'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
            if month not in wb.sheetnames:
                continue

            df = pd.read_excel(file_path, sheet_name=month, engine='openpyxl')
            df = df.dropna(how='all')

            if 'Date' not in df.columns or 'Fuel' not in df.columns:
                continue

            # Reshape 15-minute interval data
            melted = df.melt(id_vars=['Date', 'Fuel'], var_name='Time', value_name='MW')
            melted['Datetime'] = pd.to_datetime(
                melted['Date'].astype(str) + ' ' + melted['Time'].astype(str),
                errors='coerce'
            )

            melted = melted.drop(columns=['Date', 'Time'])
            melted = melted[['Datetime', 'Fuel', 'MW']]
            all_rows.append(melted)

            print(f"✅ Processed {month} in {file}")

    except Exception as e:
        print(f"❌ Failed to process {file}: {e}")

# === 4. Combine and save ===
fuel_mix_combined = pd.concat(all_rows, ignore_index=True)
fuel_mix_combined = fuel_mix_combined.dropna(subset=['Datetime'])

output_path = os.path.join(base_path, "..", "fuel_mix_combined_2016_2025.csv")
fuel_mix_combined.to_csv(output_path, index=False)

print(f"\n✅ Saved combined fuel mix to:\n{output_path}")


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1162356110.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  melted['Datetime'] = pd.to_datetime(


✅ Processed Jan in IntGenbyFuel2017.xlsx


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1162356110.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  melted['Datetime'] = pd.to_datetime(


✅ Processed Feb in IntGenbyFuel2017.xlsx


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1162356110.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  melted['Datetime'] = pd.to_datetime(


...

✅ Processed Feb in IntGenbyFuel2025.xlsx


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1162356110.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  melted['Datetime'] = pd.to_datetime(


✅ Processed Mar in IntGenbyFuel2025.xlsx


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1162356110.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  melted['Datetime'] = pd.to_datetime(


✅ Processed Apr in IntGenbyFuel2025.xlsx

✅ Saved combined fuel mix to:
C:\Users\amina.talipova\Desktop\ercot\FM\..\fuel_mix_combined_2016_2025.csv


In [16]:
import xlrd
import pandas as pd
from openpyxl import Workbook

def convert_xls_to_xlsx(xls_path, xlsx_path):
    book_xls = xlrd.open_workbook(xls_path, formatting_info=False)
    writer = pd.ExcelWriter(xlsx_path, engine='openpyxl')

    for sheet_index in range(book_xls.nsheets):
        sheet = book_xls.sheet_by_index(sheet_index)
        data = [sheet.row_values(rownum) for rownum in range(sheet.nrows)]
        df = pd.DataFrame(data)
        df.to_excel(writer, sheet_name=sheet.name, index=False, header=False)

    writer.close()

# List of .xls files with full paths
xls_folder = r"C:\Users\amina.talipova\Desktop\ercot\FM"
files = [
    "IntGenByFuel2011.xls",
    "IntGenByFuel2012.xls",
    "IntGenByFuel2013.xls",
    "IntGenByFuel2014.xls",
    "IntGenByFuel2015.xls"
]

# Convert each to .xlsx in the same folder
for file in files:
    xls_path = os.path.join(xls_folder, file)
    xlsx_path = xls_path.replace(".xls", ".xlsx")
    convert_xls_to_xlsx(xls_path, xlsx_path)
    print(f"✅ Converted: {file} → {os.path.basename(xlsx_path)}")


✅ Converted: IntGenByFuel2011.xls → IntGenByFuel2011.xlsx
✅ Converted: IntGenByFuel2012.xls → IntGenByFuel2012.xlsx
✅ Converted: IntGenByFuel2013.xls → IntGenByFuel2013.xlsx
✅ Converted: IntGenByFuel2014.xls → IntGenByFuel2014.xlsx
✅ Converted: IntGenByFuel2015.xls → IntGenByFuel2015.xlsx


In [17]:
import pandas as pd
import os
from openpyxl import load_workbook

# === 1. Define files ===
fuel_mix_files = [
    "IntGenByFuel2011.xlsx",
    "IntGenByFuel2012.xlsx",
    "IntGenByFuel2013.xlsx",
    "IntGenByFuel2014.xlsx",
    "IntGenByFuel2015.xlsx"
]

# === 2. Folder where files are stored ===
base_path = r"C:\Users\amina.talipova\Desktop\ercot\FM"

# === 3. Processing logic ===
all_rows = []
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for file in fuel_mix_files:
    file_path = os.path.join(base_path, file)
    try:
        wb = load_workbook(file_path, read_only=True)
        for sheet in wb.sheetnames:
            if not any(sheet.startswith(mon) for mon in months):
                continue

            df = pd.read_excel(file_path, sheet_name=sheet, engine='openpyxl', header=0)
            df = df.dropna(how='all')

            # Drop DST and column 0
            df = df[[col for col in df.columns if not str(col).startswith('DST') and not str(col).startswith('dst')]]
            df = df.drop(columns=[0, 'Total'], errors='ignore')

            if 'Date-Fuel' not in df.columns:
                continue

            # Split Date-Fuel into Date and Fuel
            df[['Date', 'Fuel']] = df['Date-Fuel'].astype(str).str.split('_', n=1, expand=True)
            df = df.drop(columns=['Date-Fuel'])

            # Reshape to long format
            melted = df.melt(id_vars=['Date', 'Fuel'], var_name='Time', value_name='MW')
            melted['Date'] = pd.to_datetime(melted['Date'], format='%m/%d/%y', errors='coerce')
            melted['Time'] = pd.to_numeric(melted['Time'], errors='coerce')
            melted['Datetime'] = melted['Date'] + pd.to_timedelta(melted['Time'], unit='D')
            melted = melted.drop(columns=['Date', 'Time'])

            melted = melted[['Datetime', 'Fuel', 'MW']]
            all_rows.append(melted)
            print(f"✅ Processed {sheet} in {file}")

    except Exception as e:
        print(f"❌ Failed to process {file}: {e}")

# === 4. Combine and save ===
fuel_mix_combined = pd.concat(all_rows, ignore_index=True)
fuel_mix_combined = fuel_mix_combined.dropna(subset=['Datetime'])

output_path = os.path.join(base_path, "..", "fuel_mix_combined_2011_2015.csv")
fuel_mix_combined.to_csv(output_path, index=False)

print(f"\n✅ Saved combined fuel mix to:\n{output_path}")


✅ Processed Dec12 in IntGenByFuel2012.xlsx
✅ Processed Nov12 in IntGenByFuel2012.xlsx
✅ Processed Oct12 in IntGenByFuel2012.xlsx
✅ Processed Sep12 in IntGenByFuel2012.xlsx
✅ Processed Aug12 in IntGenByFuel2012.xlsx
✅ Processed Jul12 in IntGenByFuel2012.xlsx
✅ Processed Jun12 in IntGenByFuel2012.xlsx
✅ Processed May12 in IntGenByFuel2012.xlsx
✅ Processed Apr12 in IntGenByFuel2012.xlsx
✅ Processed Mar12 in IntGenByFuel2012.xlsx
✅ Processed Feb12 in IntGenByFuel2012.xlsx
✅ Processed Jan12 in IntGenByFuel2012.xlsx
✅ Processed Dec13 in IntGenByFuel2013.xlsx
✅ Processed Nov13 in IntGenByFuel2013.xlsx
✅ Processed Oct13 in IntGenByFuel2013.xlsx
✅ Processed Sep13 in IntGenByFuel2013.xlsx
✅ Processed Aug13 in IntGenByFuel2013.xlsx
✅ Processed Jul13 in IntGenByFuel2013.xlsx
✅ Processed Jun13 in IntGenByFuel2013.xlsx
✅ Processed May13 in IntGenByFuel2013.xlsx
✅ Processed Apr13 in IntGenByFuel2013.xlsx
✅ Processed Mar13 in IntGenByFuel2013.xlsx
✅ Processed Feb13 in IntGenByFuel2013.xlsx
✅ Processed

In [28]:
import pandas as pd
import os
from openpyxl import load_workbook

# === File path ===
file_2016 = r"C:\Users\amina.talipova\Desktop\ercot\FM\IntGenByFuel2016.xlsx"
output_2016 = r"C:\Users\amina.talipova\Desktop\ercot\fuel_mix_2016_only.csv"

# === Sheets to parse ===
valid_sheets = [f"{month}16" for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

# === Process each sheet ===
rows_2016 = []
for sheet in valid_sheets:
    try:
        df = pd.read_excel(file_2016, sheet_name=sheet, engine='openpyxl')
        df = df.dropna(how='all')

        if 'Date-Fuel' not in df.columns:
            continue

        # Split 'Date-Fuel' into two columns
        df[['Date', 'Fuel']] = df['Date-Fuel'].str.split('_', expand=True)
        df = df.drop(columns=['Date-Fuel', 'Total'], errors='ignore')

        melted = df.melt(id_vars=['Date', 'Fuel'], var_name='Time', value_name='MW')
        melted['Datetime'] = pd.to_datetime(
            melted['Date'].astype(str) + ' ' + melted['Time'].astype(str),
            errors='coerce'
        )

        melted = melted.drop(columns=['Date', 'Time'])
        melted = melted[['Datetime', 'Fuel', 'MW']].dropna()
        rows_2016.append(melted)

        print(f"✅ Processed {sheet}")

    except Exception as e:
        print(f"❌ Error in {sheet}: {e}")

# === Combine and save ===
df_2016 = pd.concat(rows_2016, ignore_index=True)
df_2016.to_csv(output_2016, index=False)
print(f"\n✅ Saved parsed 2016 fuel mix to:\n{output_2016}")


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1975963893.py:28: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



✅ Processed Jan16


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1975963893.py:28: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1975963893.py:28: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



✅ Processed Feb16


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1975963893.py:28: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1975963893.py:28: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



...

✅ Processed Oct16


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1975963893.py:28: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



✅ Processed Nov16


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_34552\1975963893.py:28: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



✅ Processed Dec16

✅ Saved parsed 2016 fuel mix to:
C:\Users\amina.talipova\Desktop\ercot\fuel_mix_2016_only.csv


In [32]:
import pandas as pd

# === Load original CSVs ===
path_2011_2015 = r"C:\Users\amina.talipova\Desktop\ercot\fuel_mix_combined_2011_2015.csv"
path_2016_only = r"C:\Users\amina.talipova\Desktop\ercot\fuel_mix_2016_only.csv"
path_2017_2025 = r"C:\Users\amina.talipova\Desktop\ercot\fuel_mix_combined_2016_2025.csv"
output_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\FM_combined.csv"

df_early = pd.read_csv(path_2011_2015)
df_2016 = pd.read_csv(path_2016_only)
df_late = pd.read_csv(path_2017_2025)

# === Unify datetime formats ===
df_early['Datetime'] = pd.to_datetime(df_early['Datetime'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
df_2016['Datetime'] = pd.to_datetime(df_2016['Datetime'], errors='coerce')
df_late['Datetime'] = pd.to_datetime(df_late['Datetime'], errors='coerce')

# Drop rows with bad datetime values
df_early.dropna(subset=['Datetime'], inplace=True)
df_2016.dropna(subset=['Datetime'], inplace=True)
df_late.dropna(subset=['Datetime'], inplace=True)

# === Merge and sort
df_combined = pd.concat([df_early, df_2016, df_late], ignore_index=True)
df_combined.sort_values('Datetime', inplace=True)

# === Save
df_combined.to_csv(output_path, index=False)
print(f"✅ Combined file saved to {output_path}")


✅ Combined file saved to C:\Users\amina.talipova\Desktop\ercot\datasets\FM_combined.csv
